In [2]:
library(styleer)

# 1. 数据导入及清洗

## 1. 对抓取的龙虎榜数据进行清洗 

检测每一天的数据是否抓到最新

In [4]:
b <- fread("./data/2022-01-27.csv", encoding = "UTF-8")
b[order(-date)
    ][1:5]

buy_inst_num,date,rank_reason,sell_inst_num,stock_name,stock_symbol
<dbl>,<date>,<chr>,<dbl>,<chr>,<int>
460.27,2022-01-26,有价格涨跌幅限制的日收盘价格涨幅达到15%的证券,709.01,建研设计,301167
2009.31,2022-01-26,连续3个交易日内收盘价格跌幅较基准指数偏离值累计达到-30%的证券,9479.98,华宝股份,300741
4907.75,2022-01-26,"连续三个交易日内,涨幅偏离值累计达20%的证券",4232.23,深南股份,2417
0.00,2022-01-26,日跌幅偏离值达7%的证券,4220.17,岭南股份,2717
4406.32,2022-01-26,日跌幅偏离值达7%的证券,3039.90,千红制药,2550


合并所有数据集查看行数并导出为csv

In [5]:
DTRank <- fbread(pattern = "*.csv", encoding = "UTF-8")
DTRank <- DTRank[, file_id := NULL, 
    ][, unique(.SD)
    ][, stock_symbol := str_pad(stock_symbol, width = 6, side = "left", pad = "0")
    ][order(-date, stock_symbol), .SD
    ]
DTRank[, .N]
fwrite(DTRank, "DTRank.csv")

[1] 4660

## 2. 清洗合并各类股票数据

导入日收益率文件数据

In [5]:
trading <- fbread(pattern = "*.txt", encoding = "UTF-8", path = "./data/Trading")
trading[, .N]
trading <- trading[, file_id := NULL
    ][, setnames(.SD, c("Stkcd", "Trddt"), c("stock_symbol", "date"))]
trading[1:5]

[1] 5730194

stock_symbol,date,Clsprc,Dnvaltrd,Dsmvosd
<int>,<date>,<dbl>,<dbl>,<dbl>
1,2017-01-25,9.26,281976294,156660584
1,2017-01-26,9.33,391844286,157844843
1,2017-02-03,9.26,292617943,156660584
1,2017-02-06,9.31,480441331,157506484
1,2017-02-07,9.30,368755274,157337304


导入换手率文件数据

In [6]:
tradingDev <- fbread(pattern = "*.txt", encoding = "UTF-8", path = "./data/TradingDev")
tradingDev[, .N]
tradingDev <- tradingDev[, setnames(.SD, c('TradingDate', 'Symbol'), c('date', 'stock_symbol'))
    ][, file_id := NULL]
tradingDev[1:5]

[1] 5730194

date,stock_symbol,ShortName,Turnover
<date>,<int>,<chr>,<dbl>
2016-01-25,1,平安银行,0.00319
2016-01-26,1,平安银行,0.00549
2016-01-27,1,平安银行,0.00482
2016-01-28,1,平安银行,0.00256
2016-01-29,1,平安银行,0.00461


合并两个文件

## 3. 将龙虎榜和股票交易数据合并

In [ ]:
stkcd_feature <- tradingDev[trading, on = .(date, stock_symbol)]
stkcd_feature[, stock_symbol := str_pad(stock_symbol, 6, side = "left", pad = "0")
    ][1:5]

date,stock_symbol,ShortName,Turnover,Clsprc,Dnvaltrd,Dsmvosd
<date>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
2017-01-25,000001,平安银行,0.00180,9.26,281976294,156660584
2017-01-26,000001,平安银行,0.00249,9.33,391844286,157844843
2017-02-03,000001,平安银行,0.00186,9.26,292617943,156660584
2017-02-06,000001,平安银行,0.00305,9.31,480441331,157506484
2017-02-07,000001,平安银行,0.00235,9.30,368755274,157337304


In [ ]:
stkcd_feature <- tradingDev[trading, on = .(date, stock_symbol)]
stkcd_feature[, stock_symbol := str_pad(stock_symbol, 6, side = "left", pad = "0")
    ][1:5]

date,stock_symbol,ShortName,Turnover,Clsprc,Dnvaltrd,Dsmvosd
<date>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
2017-01-25,000001,平安银行,0.00180,9.26,281976294,156660584
2017-01-26,000001,平安银行,0.00249,9.33,391844286,157844843
2017-02-03,000001,平安银行,0.00186,9.26,292617943,156660584
2017-02-06,000001,平安银行,0.00305,9.31,480441331,157506484
2017-02-07,000001,平安银行,0.00235,9.30,368755274,157337304


In [11]:
DTanalysis <- DTRank[stkcd_feature, on = .(date, stock_symbol)]
DTanalysis[1:5]

buy_inst_num,date,rank_reason,sell_inst_num,stock_name,stock_symbol,ShortName,Turnover,Clsprc,Dnvaltrd,Dsmvosd
<dbl>,<date>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
NA,2017-01-25,NA,NA,NA,000001,平安银行,0.00180,9.26,281976294,156660584
NA,2017-01-26,NA,NA,NA,000001,平安银行,0.00249,9.33,391844286,157844843
NA,2017-02-03,NA,NA,NA,000001,平安银行,0.00186,9.26,292617943,156660584
NA,2017-02-06,NA,NA,NA,000001,平安银行,0.00305,9.31,480441331,157506484
NA,2017-02-07,NA,NA,NA,000001,平安银行,0.00235,9.30,368755274,157337304


## 4. 策略1
对机构介入的、上了龙虎榜的股，不管是否涨跌停，只要机构净买入占总成交额比＞1.5％或者机构净买额占总成交额比（A％），换手率（B％），A×B＞＝20的股；选出来，再第二天的涨跌幅±2％内的股，判第三天的涨幅。第三天累积涨跌幅±3％的股，判第四天涨幅。第五天累积涨跌幅±3％的股，判第六天涨幅。

In [12]:
DTanalysis_DT <- DTanalysis[order(stock_symbol, date), .SD # 首先根据股票和日期排序
    ][, net_buy_inst_num := buy_inst_num - sell_inst_num # 看机构净买入量
    ][, ret := Clsprc/shift(Clsprc, type = "lag", n = 1L) - 1, by = .(stock_symbol) # 计算每天的收益率
    ][, ret_2nd := shift(Clsprc, type = "lead", n = 1L)/Clsprc - 1, by = .(stock_symbol)# 计算第2天的收益率
    ][, ret_3rd := shift(Clsprc, type = "lead", n = 2L)/shift(Clsprc, type = "lead", n = 1L) - 1, by = .(stock_symbol)# 计算第3天的收益率
    ][, ret_4th := shift(Clsprc, type = "lead", n = 3L)/shift(Clsprc, type = "lead", n = 2L) - 1, by = .(stock_symbol)# 计算第4天的收益率
    ][, ret_5th := shift(Clsprc, type = "lead", n = 4L)/shift(Clsprc, type = "lead", n = 3L) - 1, by = .(stock_symbol)# 计算第5天的收益率
    ][, ret_6th := shift(Clsprc, type = "lead", n = 5L)/shift(Clsprc, type = "lead", n = 4L) - 1, by = .(stock_symbol)# 计算第6天的收益率
    ][, ret_7th := shift(Clsprc, type = "lead", n = 6L)/shift(Clsprc, type = "lead", n = 5L) - 1, by = .(stock_symbol)# 计算第7天的收益率       
    ][!is.na(buy_inst_num), .SD
    ]
DTanalysis_DT[1:5]

buy_inst_num,date,rank_reason,sell_inst_num,stock_name,stock_symbol,ShortName,Turnover,Clsprc,Dnvaltrd,Dsmvosd,net_buy_inst_num,ret,ret_2nd,ret_3rd,ret_4th,ret_5th,ret_6th,ret_7th
<dbl>,<date>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
8658.03,2017-07-11,日涨幅偏离值达7%的证券,0.00,平安银行,000001,平安银行,0.02253,10.25,3842010171,173409394,8658.03,0.06882169,0.008780488,0.054158607,0.000000000,-0.008256881,0.022201665,0.00361991
28154.20,2021-02-25,日涨幅偏离值达7%的证券,22422.10,万科A,000002,万科A,0.02671,32.99,8382663604,320582082,5732.10,0.10003334,0.003334344,0.007552870,-0.001799100,0.006308201,-0.024179104,-0.03517895
769.45,2021-06-09,"连续三个交易日内,涨幅偏离值累计达20%的证券",788.98,国华网安,000004,国华网安,0.11303,19.82,246709411,2281790,-19.53,0.09988901,0.099899092,0.100000000,-0.100083403,-0.066265060,0.038213400,-0.05449331
0.00,2019-09-03,日涨幅偏离值达7%的证券,527.69,世纪星源,000005,世纪星源,0.09123,3.48,330930683,3681653,-527.69,0.10126582,0.100574713,-0.036553525,-0.035230352,-0.002808989,-0.008450704,-0.01136364
0.00,2020-10-14,日跌幅偏离值达7%的证券,1080.73,深振业A,000006,深振业A,0.05205,6.39,462640390,8615688,-1080.73,-0.10000000,-0.035993740,0.006493506,0.003225806,-0.006430868,-0.012944984,0.00000000
